In [ ]:
import itertools
import spacy
import random
import os
from spacy.util import minibatch, compounding
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
root_path = 'gdrive/My Drive/AI&Ethics/'
os.chdir(root_path)

Mounted at /content/gdrive


In [ ]:
!pip install ekphrasis
!pip install vaderSentiment
!pip install textstat
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import pickle
import sys
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch
import copy
from transformers import BertModel, RobertaModel, BertTokenizer, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
import json
from sklearn.metrics import accuracy_score
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
model_path = 'bert-base-uncased'
# model_path = 'monsoon-nlp/tamillion'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# New Section

In [ ]:
from datasets import list_datasets, load_dataset
dataset = load_dataset('hatexplain', split = 'train')

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/802fcd855438812094e336cea509c99b04b890e4e0846c0385877ee2c7361e93)


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [ ]:
def get_ids(criteria):
    if criteria == 'Race':
        column = 2
    elif criteria == 'Country':
        column = 3
    elif criteria == 'Age':
        column = 4
    elif criteria == 'Gender':
        column = 5
    elif criteria == 'Religion':
        column = 6
    
    ids = {}
    with open(annotator_file) as annotfile:
        next(annotfile)
        annots = annotfile.read().splitlines()
        for a in annots:
            annot = a.split(",")
            try:
                ids[annot[column]].append(annot[1])
            except KeyError:
                ids[annot[column]] = [annot[1]]
    return ids

def get_freq(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return len(ids[cl])
    return 200

def get_annotator_class(id, ids):
    for cl in ids.keys():
        if str(id) in ids[cl]:
            return cl
    return None

ids = get_ids('Religion')
print(type(ids))
# print(ids_1)

<class 'dict'>


In [ ]:
data_file = 'data/greedy_approach_annotators.json'

df = pd.read_json(data_file)
print(df.head())
print(len(dataset), len(df))
classes = ['Race', 'Country', 'Religion', 'Gender']
for class_name in classes:
    ids = get_ids(class_name)
    info_dict = {}
    for key in ids:
        for val in ids[key]:
            info_dict[val] = key
    class_data = []
    for id in df['annotator_id']: class_data.append(info_dict[str(id)])
    df[class_name] = class_data
print(df['text'])
df.to_csv('data/annotators_final.csv', index=False)

   label  ...                                               text
0      2  ...  [u, really, think, i, would, not, have, been, ...
1      2  ...  [if, english, is, not, imposition, then, hindi...
2      2  ...  [no, liberal, congratulated, hindu, refugees, ...
3      2  ...  [he, said, bro, even, your, texts, sound, redn...
4      1  ...  [i, live, and, work, with, many, legal, mexica...

[5 rows x 4 columns]
15383 14242
0        [u, really, think, i, would, not, have, been, ...
1        [if, english, is, not, imposition, then, hindi...
2        [no, liberal, congratulated, hindu, refugees, ...
3        [he, said, bro, even, your, texts, sound, redn...
4        [i, live, and, work, with, many, legal, mexica...
                               ...                        
14237    [thanks, for, coming, to, my, ted, talk, p.s.,...
14238    [<user>, <user>, iran, has, the, 2, n, biggest...
14239    [or, maybe, those, were, not, meant, to, be, h...
14240            [good, morning, ados, black,

# Run these

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.stem.porter import *
import string
import re

stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()


def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    #tokens = re.split("[^a-zA-Z]*", tweet.lower())
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()

vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)
%load_ext autoreload
%autoreload 2

In [ ]:
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)
    return df_train, df_val, df_test

In [ ]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *

sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA, FRE,syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

other_features_names = ["FKRA", "FRE","num_syllables", "avg_syl_per_word", "num_chars", "num_chars_total", \
                        "num_terms", "num_words", "num_unique_words", "vader neg","vader pos","vader neu", "vader compound", \
                        "num_hashtags", "num_mentions", "num_urls", "is_retweet"]

#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

In [ ]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
df = pd.read_csv('data/annotators_race.csv')
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
len(df_train), len(df_val), len(df_test)

try:
    with open('results_individual_ml.pkl', 'rb') as f:
        results = pickle.load(f)
except:
    results = {}
    with open('results_individual_ml.pkl', 'wb') as f:
        pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

{}


In [ ]:
for c1 in class_names:
    if c1 in results[parameter]: continue
    else: results[parameter][c1] = {}
    for c2 in class_names:
        train_df = df_train[df_train[parameter] == c1]
        test_df = df_test[df_test[parameter] == c2]

        vectorizer = TfidfVectorizer(
            #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
            tokenizer=tokenize,
            preprocessor=preprocess,
            ngram_range=(1, 3),
            stop_words=stopwords, #We do better when we keep stopwords
            use_idf=True,
            smooth_idf=False,
            norm=None, #Applies l2 norm smoothing
            decode_error='replace',
            max_features=10000,
            min_df=5,
            max_df=0.501
            )

        #Construct tfidf matrix and get relevant scores
        tweets_train = list(train_df['text'])
        tweets_test = list(test_df['text'])
        tfidf_transform = vectorizer.fit(tweets_train)
        vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
        idf_vals = vectorizer.idf_
        idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores

        tweet_tags_train = []
        for t in tweets_train:
            tokens = basic_tokenize(preprocess(t))
            tags = nltk.pos_tag(tokens)
            tag_list = [x[1] for x in tags]
            #for i in range(0, len(tokens)):
            tag_str = " ".join(tag_list)
            tweet_tags_train.append(tag_str)

        tweet_tags_test = []
        for t in tweets_test:
            tokens = basic_tokenize(preprocess(t))
            tags = nltk.pos_tag(tokens)
            tag_list = [x[1] for x in tags]
            #for i in range(0, len(tokens)):
            tag_str = " ".join(tag_list)
            tweet_tags_test.append(tag_str)

        #We can use the TFIDF vectorizer to get a token matrix for the POS tags
        pos_vectorizer = TfidfVectorizer(
            #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
            tokenizer=None,
            lowercase=False,
            preprocessor=None,
            ngram_range=(1, 3),
            stop_words=None, #We do better when we keep stopwords
            use_idf=False,
            smooth_idf=False,
            norm=None, #Applies l2 norm smoothing
            decode_error='replace',
            max_features=5000,
            min_df=5,
            max_df=0.501,
            )

        pos_transform = pos_vectorizer.fit(pd.Series(tweet_tags_train))
        pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}

        tfidf_train = tfidf_transform.transform(tweets_train).toarray()
        tfidf_test = tfidf_transform.transform(tweets_test).toarray()

        pos_train = pos_transform.transform(pd.Series(tweet_tags_train)).toarray()
        pos_test = pos_transform.transform(pd.Series(tweet_tags_test)).toarray()

        feats_train = get_feature_array(tweets_train)
        feats_test = get_feature_array(tweets_test)

        M_train = np.concatenate([tfidf_train,pos_train,feats_train],axis=1)
        M_test = np.concatenate([tfidf_test,pos_test,feats_test],axis=1)

        X_train = pd.DataFrame(M_train)
        y_train = train_df['label'].astype(int)
        X_test = pd.DataFrame(M_test)
        y_test = test_df['label'].astype(int)

        from sklearn.linear_model import LogisticRegression
        from sklearn.feature_selection import SelectFromModel
        from sklearn.metrics import classification_report
        from sklearn.svm import LinearSVC

        select = SelectFromModel(LogisticRegression(class_weight='balanced',penalty="l2",C=0.01))
        transform = select.fit(X_train,y_train)

        X_train_ = transform.transform(X_train)
        X_test_ = transform.transform(X_test)

        model = LinearSVC(class_weight='balanced',C=0.01, penalty='l2', loss='squared_hinge',multi_class='ovr').fit(X_train_, y_train)

        y_preds_train = model.predict(X_train_)
        y_preds_test = model.predict(X_test_)

        f1 = f1_score(y_test, y_preds_test, average='macro')
        acc = accuracy_score(y_test, y_preds_test)

        if c1 not in results[parameter]: results[parameter][c1] = {}

        results[parameter][c1][c2] = {'f1': f1, 'acc':acc}
        with open('results_individual_ml.pkl', 'wb') as f:
            pickle.dump(results, f)
        print(c1, c2, f1, acc)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
 
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat
 
def evaluate(test_dataloader, model):
    model.eval()
    y_preds, y_test = np.array([]), np.array([])

    for batch in test_dataloader:
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device).long()
        with torch.no_grad():        
            ypred = model(b_input_ids, b_input_mask)
        ypred = ypred[0].cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_preds = np.hstack((y_preds, get_predicted(ypred)))
        y_test = np.hstack((y_test, label_ids))

    macro_f1 = f1_score(y_test, y_preds, average='macro')
    report = classification_report(y_test, y_preds)
    print(report)
    return macro_f1, y_preds, y_test
 
def train(training_dataloader, validation_dataloader, model, filepath = None, weights = None, learning_rate = 2e-5, epochs = 1, print_every = 10):
    total_steps = len(training_dataloader) * epochs
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    best_weighted_f1 = 0
    best_model = None
    # current_epoch, best_weighted_f1 = load_metrics(filepath, model, optimizer)
    if weights == None:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.CrossEntropyLoss(weight=weights)
    for epoch_i in tqdm(range(0, epochs)):
        model.train()
        for step, batch in enumerate(training_dataloader):
            b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device).long()
            
            outputs = model(b_input_ids, b_input_mask)
            loss = criterion(outputs[0], b_labels)
 
            if step%print_every == 0:
                print(loss.item())
 
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
 
        print('### Validation Set Stats')
        weighted_f1, ypred, ytest = evaluate(validation_dataloader, model)
        print("  Macro F1: {0:.3f}".format(weighted_f1))
        if weighted_f1 > best_weighted_f1:
            best_weighted_f1 = weighted_f1
            best_model = model
            # save_metrics(filepath, epoch_i, model, optimizer, weighted_f1)
        
    return best_model

In [ ]:
class_names = ['Caucasian', 'Hispanic', 'Indian', 'African']
parameter = 'Race'
df = pd.read_csv('data/annotators_race.csv')
df_train, df_val, df_test = split_stratified_into_train_val_test(df, stratify_colname=parameter,
                                         frac_train=0.8, frac_val=0.1, frac_test=0.1,
                                         random_state=42)
# len(df_train), len(df_val), len(df_test)

# try:
#     with open('results_individual.pkl', 'rb') as f:
#         results = pickle.load(f)
# except:
results = {}
with open('results_individual.pkl', 'wb') as f:
    pickle.dump(results, f)
print(results)
if parameter not in results: results[parameter] = {}

{}


In [ ]:
for c1 in class_names:
    if c1 in results[parameter]: continue
    else: results[parameter][c1] = {}
    train_df = df_train[df_train[parameter] == c1]
    val_df = df_val[df_val[parameter] == c1]
    train_data, val_data = Dataset(train_df), Dataset(val_df)
    model = SC_weighted_BERT(model_path).to(device)
    model = train(train_data.DataLoader, val_data.DataLoader, model, None, epochs = 5)
    for c2 in class_names:
        test_df = df_test[df_test[parameter] == c2]
        test_data = Dataset(test_df)
        f1, ypreds, ytest = evaluate(test_data.DataLoader, model)
        acc = accuracy_score(ytest, ypreds)
        results[parameter][c1][c2] = {'f1': f1, 'acc':acc}
    with open('results_individual.pkl', 'rb') as f:
        results = pickle.load(f)
    with open('results_individual.pkl', 'wb') as f:
        pickle.dump(results, f)
    print(c1, results[c1])

5031
629


  0%|          | 0/5 [00:00<?, ?it/s]

1.1332472562789917
1.032253384590149
1.0149624347686768
1.017764925956726
0.8575710654258728
0.7544137239456177
0.8850621581077576
0.8717197179794312
0.8048531413078308
0.7126165628433228
0.7826817631721497
0.7599096298217773
0.6364507675170898
0.8325504064559937
0.799396276473999
0.6170777678489685
### Validation Set Stats
